# Test Variational Inference algorithms for Wold processes

In this notebook, we test the various variational inference (VI) algorithms implemented for the multivariate Wold process. Namely:

* `WoldModelVariational`: VI with parameters $\alpha$ and $\beta$
* `WoldModelVariationalFixedBeta`: VI with parameters $\alpha$ only ($\beta$ are fixed hyper-parameters)

---

Load extensions for debugging

In [1]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

Import the libraries of interest for this notebook

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

import itertools
import torch
import numpy as np

# Set numpy print format
np.set_printoptions(precision=2, floatmode='fixed', sign=' ')

import tsvar

---

## Generate small toy example dataset


In [9]:
# Define random parameters
dim = 2  # Dimensionality of the process
end_time = 5e4  # Choose a long observation window
mu = torch.tensor([0.3, 0.1])
beta = torch.tensor([
    [1.0, 0.2],
    [0.5, 0.1]
])
# Use the same constraint as GrangerBusca to allow fair comparison
alpha = torch.tensor([
    [0.7, 0.3],
    [0.0, 1.0]
])
coeffs_true = torch.cat((mu, beta.flatten(), alpha.flatten())).numpy()
print('  - Simulate lots of data...')
# Simulate lots of data
wold_sim = tsvar.simulate.MultivariateWoldSimulator(
    mu_a=mu, alpha_ba=alpha, beta_ba=beta)
events = wold_sim.simulate(max_jumps=50e3, seed=4243)
events = [torch.tensor(ev, dtype=torch.float) for ev in events]
print((f"    - Simulated {sum(map(len, events)):,d} events "
       f"with end time: {end_time}"))
print("    - Events:")
print("      - dim 0:", events[0])
print("      - dim 1:", events[1])

  - Simulate lots of data...
    - Simulated 50,000 events with end time: 50000.0
    - Events:
      - dim 0: tensor([5.3977e+00, 7.1193e+00, 7.3136e+00,  ..., 4.3349e+04, 4.3350e+04,
        4.3357e+04])
      - dim 1: tensor([1.8407e+01, 2.2158e+01, 2.2301e+01,  ..., 4.3358e+04, 4.3358e+04,
        4.3360e+04])


---

## Test Mean-Field VI with Fixed $\{\beta\}$s using `WoldModelVariationalFixedBeta`

Define the model object.

In [ ]:
beta_ji_plus_one = np.repeat(beta[:,np.newaxis], dim, axis=1) + 1
beta_ji_plus_one = np.vstack((np.zeros(dim), beta_ji_plus_one))
beta_ji_plus_one

In [ ]:
model_fixed_beta = tsvar.models.WoldModelVariationalFixedBeta(verbose=True)
model_fixed_beta.observe(events, beta=beta_ji_plus_one)

Set the hyper-parameters and fit the model.

In [ ]:
as_pr = 1.0 * np.ones((dim + 1, dim))
ar_pr = 1.0 * np.ones((dim + 1, dim))

zc_pr = [1.0 * np.ones((len(events[i]), dim+1)) for i in range(dim)]

callback = tsvar.utils.callbacks.LearnerCallbackMLE(
    x0=(as_pr / ar_pr).flatten(), 
    print_every=10, 
    coeffs_true=np.hstack((mu, alpha.flatten())),
    acc_thresh=0.05, dim=dim)

model_fixed_beta.fit(as_pr=as_pr, ar_pr=ar_pr, zc_pr=zc_pr, max_iter=200, tol=1e-5, callback=callback)

Explore the results.

In [ ]:
alpha_hat_mode = np.round((model_fixed_beta._as_po >= 1) * (model_fixed_beta._as_po - 1) / model_fixed_beta._ar_po, 2)
alpha_hat_mean = np.round(model_fixed_beta._as_po / model_fixed_beta._ar_po, 2)

baseline_hat = alpha_hat_mean[0,:]
adjacency_hat = alpha_hat_mean[1:,:]

print('Baseline:')
print('---------')
print('Ground truth:')
print(mu.numpy())
print('Estimated:')
print(baseline_hat)
print()

print('Adjacency:')
print('---------')
print('Ground truth:')
print(alpha.numpy())
print('Estimated:')
print(adjacency_hat)
print()

In [ ]:
x_iter = callback.history['iter']
y_base = [tsvar.utils.metrics.relerr(np.array(coeffs[:3]), baseline) for coeffs in callback.history['coeffs']]

plt.plot(x_iter, y_base);

In [ ]:
x_iter = callback.history['iter']
y_acc = [tsvar.utils.metrics.accuracy(adj_test=np.array(coeffs[-9:]), 
                                      adj_true=adjacency.flatten(), 
                                      threshold=0.02) 
         for coeffs in callback.history['coeffs']]

plt.plot(x_iter, y_acc);

---

## Test Mean-Field VI with Variable $\{\beta\}$s using `WoldModelVariational`

Create model object and set the data.

In [4]:
# Set model
model = tsvar.models.WoldModelVariational(verbose=True)
model.observe(events)

Define the parameters of the prior.

In [5]:
# Set priors
# prior: Alpha
as_pr = 1.0 * np.ones((dim + 1, dim))
ar_pr = 1.0 * np.ones((dim + 1, dim))
# prior: Beta
bs_pr = 100.0 * np.ones((dim, dim))
br_pr = 100.0 * np.ones((dim, dim))
# prior: Z
zc_pr = [1.0 * np.ones((len(events[i]), dim+1)) for i in range(dim)]

Fit the model.

In [6]:
# Set callback
coeffs_start = (as_pr / ar_pr).flatten()
callback = tsvar.utils.callbacks.LearnerCallbackMLE(x0=coeffs_start,
                                                    print_every=10,
                                                    coeffs_true=alpha.numpy().flatten(),
                                                    acc_thresh=0.015,
                                                    dim=dim)
# Fit model
model.fit(as_pr=as_pr, ar_pr=ar_pr, bs_pr=bs_pr, br_pr=br_pr, zc_pr=zc_pr, 
          max_iter=1000, tol=1e-5, callback=callback)

################################################################################
0
[[-1.78e+00 -1.77e+00 -1.77e+00]
 [-1.78e+00 -1.77e+00 -1.77e+00]
 [-1.78e+00 -1.77e+00 -1.77e+00]
 ...
 [-1.78e+00 -1.77e+00 -1.77e+00]
 [-1.78e+00 -1.77e+00 -1.00e+08]
 [-1.78e+00 -1.77e+00 -1.00e+08]]
-100000001.77399375 -1.77339891935633
1
[[-1.65e+00 -1.65e+00 -1.65e+00]
 [-1.65e+00 -1.65e+00 -1.65e+00]
 [-1.65e+00 -1.65e+00 -1.00e+08]
 ...
 [-1.65e+00 -1.00e+08 -1.65e+00]
 [-1.65e+00 -1.00e+08 -1.65e+00]
 [-1.65e+00 -1.00e+08 -1.65e+00]]
-100000001.64751987 -1.6469432535479758
################################################################################
0
[[-1.34e+00 -2.06e+00 -2.11e+00]
 [-1.34e+00 -2.06e+00 -2.11e+00]
 [-1.34e+00 -2.06e+00 -2.11e+00]
 ...
 [-1.34e+00 -2.06e+00 -2.11e+00]
 [-1.34e+00 -2.06e+00 -2.11e+00]
 [-1.34e+00 -2.06e+00 -1.00e+08]]
-100000002.11406474 -1.3443640658312415
1
[[-1.26e+00 -2.00e+00 -1.83e+00]
 [-1.26e+00 -2.00e+00 -1.83e+00]
 [-1.26e+00 -2.00e+00 -1.00e+08]
 

################################################################################
0
[[-0.99 -2.50 -2.91]
 [-0.99 -2.50 -2.91]
 [-0.99 -2.50 -2.91]
 ...
 [-0.99 -2.50 -2.91]
 [-0.99 -2.50 -2.91]
 [-0.99 -2.50 -2.91]]
-100000002.91028915 -0.9855124050797777
1
[[-9.56e-01 -2.57e+00 -2.14e+00]
 [-9.56e-01 -2.57e+00 -2.14e+00]
 [-9.56e-01 -2.57e+00 -2.14e+00]
 ...
 [-9.56e-01 -2.57e+00 -2.14e+00]
 [-9.56e-01 -2.57e+00 -2.14e+00]
 [-9.56e-01 -1.00e+08 -2.14e+00]]
-100000002.57398665 -0.9563884372012037
################################################################################
0
[[-0.98 -2.50 -2.92]
 [-0.98 -2.50 -2.92]
 [-0.98 -2.50 -2.92]
 ...
 [-0.98 -2.50 -2.92]
 [-0.98 -2.50 -2.92]
 [-0.98 -2.50 -2.92]]
-100000002.92183436 -0.9829594353197706
1
[[-9.55e-01 -2.58e+00 -2.14e+00]
 [-9.55e-01 -2.58e+00 -2.14e+00]
 [-9.55e-01 -2.58e+00 -2.14e+00]
 ...
 [-9.55e-01 -2.58e+00 -2.14e+00]
 [-9.55e-01 -2.58e+00 -2.14e+00]
 [-9.55e-01 -1.00e+08 -2.14e+00]]
-100000002.57982397 -0.954586919197611

KeyboardInterrupt: 

In [13]:
alpha_hat_mean = np.round(model._as_po / model._ar_po, 2)

baseline_hat = alpha_hat_mean[0,:]
print('Baseline:')
print('---------')
print('Ground truth:')
print(mu.numpy())
print('Estimated:')
print(baseline_hat)
print()

adjacency_hat = alpha_hat_mean[1:,:]
print('Adjacency:')
print('---------')
print('Ground truth:')
print(alpha.numpy())
print('Estimated:')
print(adjacency_hat)
print()

beta_hat = np.round(model._br_po / (model._bs_po - 1), 2) - 1
print('Beta:')
print('-----')
print('Ground truth:')
print(beta.numpy() * np.ones((dim, dim)))
print('Estimated:')
print(beta_hat)

Baseline:
---------
Ground truth:
[ 0.30  0.10]
Estimated:
[ 0.17  0.19]

Adjacency:
---------
Ground truth:
[[ 0.70  0.30]
 [ 0.00  1.00]]
Estimated:
[[ 0.30  0.32]
 [ 0.27  0.35]]

Beta:
-----
Ground truth:
[[ 1.00  0.20]
 [ 1.00  0.20]]
Estimated:
[[ 0.77  0.66]
 [ 0.57  0.81]]
